In [1]:
import numpy as np
import pandas as pd
import sys
import random
import progressbar
import torch
import pickle
from mytree import *
from utils import *
from treeUtil import *
import tqdm
import argparse
import torch.nn as nn
from torch.autograd import Variable
import torch.nn.functional as F
from torch.nn.utils import clip_grad_norm_
import functools
from sklearn.linear_model import LogisticRegression
import gensim
from gensim.models import KeyedVectors
import string
from stanfordcorenlp import StanfordCoreNLP
import nltk.tree
from ast import literal_eval
import pptree
import logging

In [2]:
from nltk.tokenize import word_tokenize
import re
import os
from sklearn.utils import resample
from sklearn.preprocessing import LabelEncoder
from logger import Logger

/Library/Frameworks/Python.framework/Versions/3.7/lib/python3.7/site-packages/tensorflow/python/framework/dtypes.py:517: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint8 = np.dtype([("qint8", np.int8, 1)])
/Library/Frameworks/Python.framework/Versions/3.7/lib/python3.7/site-packages/tensorflow/python/framework/dtypes.py:518: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_quint8 = np.dtype([("quint8", np.uint8, 1)])
/Library/Frameworks/Python.framework/Versions/3.7/lib/python3.7/site-packages/tensorflow/python/framework/dtypes.py:519: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint16 = np.dtype([("qint16", np.int16, 1)])
/

In [3]:
import spacy
from spacy import displacy
from collections import Counter
import en_core_web_sm
from spacy.pipeline import merge_entities
nlp = en_core_web_sm.load()
nlp.add_pipe(merge_entities)

In [4]:
import copy
from sklearn.utils import shuffle
from sklearn import metrics

In [23]:
proanti = True # True if doing pro/anti (ideology classification); False if doing nuetral/non-neutral classification (Step1: Stance Detection)
w2vec = False # False if in-domain; True if general
ner = True # if replacing named entities with their entity tags
blackout = False # if blacking out the named entities
balanced = True # if dataset has to be balanced or not
undersample = True # True if undersampling, False if oversampling
non_trainable = True # if first layer of classifier is fixed
economic = True # economic or tech

In [24]:
# Create a namecode for the model with set parameters
dic = {0:'ProAnti', 1:'General', 2:'NER', 3:'Blackout', 4:'Balance', 5:'Undersample', 6:'Fixed'}
bool_list = [proanti, w2vec, ner, blackout, balanced, undersample, non_trainable]
if economic:
    policy = 'economic'
else:
    policy = 'tech'
corpus_path = '../data/new/'+policy
namecode = policy+'_w2v'
for index, bb in enumerate(bool_list):
    namecode += '_'
    if bb:
        namecode += '1'
    else:
        namecode += '0'
    namecode += dic[index]
namecode

'economic_w2v_1ProAnti_0General_1NER_0Blackout_1Balance_1Undersample_1Fixed'

In [7]:
#For logging
import logging

#Remove all the previous handlers
for handler in logging.root.handlers[:]:
    logging.root.removeHandler(handler)

#Create the file for logging purposes -> CHANGE THE LEVEL TYPE TO logging.DEBUG when debugging or finding faults
logging.basicConfig(filename='log_'+namecode+'.log',
                            filemode='a',
                            format='%(asctime)s,%(msecs)d %(name)s %(levelname)s %(message)s',
                            datefmt='%m/%d/%Y %I:%M:%S %p',
                            level=logging.INFO)

In [8]:
logger = Logger('./logs_'+namecode)

In [9]:
class RecursiveNN(nn.Module):
    def __init__(self, word_embeddings, vocab, embedSize=300, numClasses=2, beta = 0.3, use_weight = True, non_trainable = non_trainable):
        super(RecursiveNN, self).__init__()
        self.embedding = nn.Embedding.from_pretrained(word_embeddings)
        self.embedding.weight.requires_grad = True
        if non_trainable:
            self.embedding.weight.requires_grad = False
        else:
            self.embedding = nn.Embedding(len(vocab), embedSize)
        self.embedding = nn.Embedding(len(vocab), embedSize)
        self.W = nn.Linear(2*embedSize, embedSize, bias=True)
        self.nonLinear = torch.tanh
        self.projection = nn.Linear(embedSize, numClasses, bias=True)
        self.nodeProbList = []
        self.labelList = []
        self.loss = Var(torch.FloatTensor([0]))
        self.V = vocab
        self.beta = beta
        self.use_weight = use_weight
        self.total_rep = None #
        self.count_rep = 0 #
        self.numClasses = numClasses

    def traverse(self, node):
        if node.isLeaf:
            if node.getLeafWord() in self.V:  # check if right word is in vocabulary
                word = node.getLeafWord()
            else:  # otherwise use the unknown token
                word = 'UNK'
            # print(self.V[word],len(self.V),word,(torch.LongTensor([int(self.V[word])])))
            currentNode = (self.embedding(Var(torch.LongTensor([int(self.V[word])]))))
        else: currentNode = self.nonLinear(self.W(torch.cat((self.traverse(node.left),self.traverse(node.right)),1)))
        currentNode = currentNode/(torch.norm(currentNode))
        assert node.label!=None
        self.nodeProbList.append(self.projection(currentNode))
        # print (node.label)
        self.labelList.append(torch.LongTensor([node.label]))
        loss_weight = 1-self.beta if node.annotated else self.beta
        self.loss += (loss_weight*F.cross_entropy(input=torch.cat([self.projection(currentNode)]),target=Var(torch.cat([torch.LongTensor([node.label])]))))
        if not node.isRoot():
            if self.total_rep is None:
                self.total_rep = currentNode.data.clone()
            else:
                self.total_rep += currentNode.data.clone()
            self.count_rep += 1
        return currentNode   
    
    def traverse_test(self, node):
        if node.isLeaf:
            if node.getLeafWord() in self.V:  # check if right word is in vocabulary
                word = node.getLeafWord()
            else:  # otherwise use the unknown token
                word = 'UNK'
            # print(self.V[word],len(self.V),word,(torch.LongTensor([int(self.V[word])])))
            currentNode = (self.embedding(Var(torch.LongTensor([int(self.V[word])]))))
        else: currentNode = self.nonLinear(self.W(torch.cat((self.traverse_test(node.left),self.traverse_test(node.right)),1)))
        currentNode = currentNode/(torch.norm(currentNode))
#         assert node.label!=None
        self.nodeProbList.append(self.projection(currentNode))
        loss_weight = 1-self.beta if node.annotated else self.beta

        if not node.isRoot():
            if self.total_rep is None:
                self.total_rep = currentNode.data.clone()
            else:
                self.total_rep += currentNode.data.clone()
            self.count_rep += 1
        return currentNode

    def forward(self, x):
        self.nodeProbList = []
        self.labelList = []
        self.loss = Var(torch.FloatTensor([0]))
        self.traverse(x)
        self.labelList = Var(torch.cat(self.labelList))
        return torch.cat(self.nodeProbList)

    def getLoss(self, tree):
        nodes = self.forward(tree)
        predictions = nodes.max(dim=1)[1]
        loss = self.loss
        return predictions,loss

    def getRep(self, tree):
        self.count_rep = 0
        self.total_rep = None
        self.nodeProbList = []
        self.labelList = []
        self.loss = Var(torch.FloatTensor([0]))

        root_rep = self.traverse(tree)

        return (torch.cat((root_rep,self.total_rep/self.count_rep),1)).data.numpy().T.flatten()

        
    def predict(self, trees):
        pbar = progressbar.ProgressBar(widgets=widgets, maxval=len(trees)).start()
        preds = []
        for j, tree in enumerate(trees):
            nodes = self.forward(tree.root)
            predictions = nodes.max(dim=1)[1]
#                     print((predictions.cpu().data).numpy(),(predictions.cpu().data).numpy().shape)
#                     print((self.labelList.cpu().data).numpy(), (self.labelList.cpu().data).numpy().shape)
            preds.append(predictions)
            pbar.update(j)
        pbar.finish()
        return preds


    def evaluate(self, trees):
            pbar = progressbar.ProgressBar(widgets=widgets, maxval=len(trees)).start()
            n = nAll = correctRoot = correctAll = 0.0
            tp = [1e-2]*self.numClasses
            fp = [1e-2]*self.numClasses
            fn = [1e-2]*self.numClasses
            f1 = [0.]*self.numClasses
            for j, tree in enumerate(trees):
                predictions,_ = self.getLoss(tree.root)
#                     print((predictions.cpu().data).numpy(),(predictions.cpu().data).numpy().shape)
#                     print((self.labelList.cpu().data).numpy(), (self.labelList.cpu().data).numpy().shape)
                correct = ((predictions.cpu().data).numpy()==(self.labelList.cpu().data).numpy())
#                     print(correct)
                correctAll += correct.sum()
                nAll += np.shape(correct.squeeze())[0] if np.size(correct)!=1 else 1 
                correctRoot += correct.squeeze()[-1] if np.size(correct)!=1 else correct[-1]
#                     print(correct.squeeze()[-1] if np.size(correct)!=1 else correct[-1])
#                     print('actual: {}'.format(tree.root.label))
                for i in range(self.numClasses):
                    size = np.size((predictions.cpu().data).numpy())
                    if size!=1:
                        pred = (predictions.cpu().data).numpy().squeeze()[-1]
                        actual = (self.labelList.cpu().data).numpy().squeeze()[-1]
                    else:
                        pred = (predictions.cpu().data).numpy()[-1]
                        actual = (self.labelList.cpu().data).numpy()[-1]
                    if pred==i and actual==i:
                        tp[i]+=1
                    elif pred==i and actual!=i:
                        fn[i]+=1
                    elif pred==i and actual!=i:
                        fp[i]+=1
                n += 1
                pbar.update(j)
#             print(tp,fp,fn)
            for i in range(self.numClasses):
                p =(1.0*tp[i]/(tp[i]+fp[i]))
                r =(1.0*tp[i]/(tp[i]+fn[i]))
                f1[i] = (2*p*r)/(p+r)
            pbar.finish()
            return correctRoot / n, correctAll/nAll, f1

    def eval_sent_lvl(self,trees,clf):
        pbar = progressbar.ProgressBar(widgets=widgets, maxval=len(trees)).start()
        n = nAll = correctRoot = correctAll = 0.0
        X_predict = []
        Y_gold = []
        for j, tree in enumerate(trees):
            tree_rep = model.getRep(tree.root)
            X_predict.append(tree_rep)
            Y_gold.append(tree.root.label)
        acc = clf.score(np.array(X_predict),np.array(Y_gold))
        return acc

## Data Preprocessing

Import the following:
- file containing dependency trees of sentences to be classified (trees can be createdd using make-trees)
- dataframe containing information about the text and label of the corresponding trees

In [10]:
file_trees = 'trees.pkl'
df_train = pickle.load(open("./trees/df_"+file_trees, 'rb'))
df_train.head() 

,tokens,target
0,"[Modi, said, the, banks, should, redouble, eff...",0
1,"[We, have, been, able, to, achieve, this, spee...",1
2,"[Acknowledging, slip, ups, in, the, Aadhaar, e...",0
3,"[Jaitley, also, said, the, Aadhaar, should, be...",1


Create dictionary for words in the dataset

In [11]:
all_tokens = set(sum(df_train['tokens'],[]))
word2idx = {}
word2idx['UNK']=0
i = 1
for token in list(all_tokens):
    word2idx[token] = i
    i+=1
idx2word = {v: k for k, v in word2idx.items()}

In [12]:
idx2word

{0: 'UNK',
 1: 'the',
 2: 'Aadhar',
 3: 'PERSON',
 4: 'fingerprints',
 5: 'and',
 6: 'achieve',
 7: 'been',
 8: 'Acknowledging',
 9: 'model',
 10: 'Aadhaar',
 11: 'in',
 12: 'an',
 13: 'be',
 14: 'registrar',
 15: 'instead',
 16: 'said',
 17: 's',
 18: 'director',
 19: 'needs',
 20: 'to',
 21: 'this',
 22: 'scalable',
 23: 'Jaitley',
 24: 'returns',
 25: 'evasions',
 26: 'of',
 27: 'also',
 28: 'as',
 29: 'due',
 30: 'deputy',
 31: 'applicant',
 32: 'financial',
 33: 'enrolment',
 34: 'speed',
 35: 'cases',
 36: 'seeding',
 37: 'redouble',
 38: 'told',
 39: 'curb',
 40: 'technical',
 41: 'UIDAI',
 42: 'there',
 43: 'tax',
 44: 'multiple',
 45: 'improve',
 46: 'scale',
 47: 'operator',
 48: 'had',
 49: 'literacy',
 50: 'should',
 51: 'toi',
 52: 'where',
 53: 'linked',
 54: 'accounts',
 55: 'with',
 56: 'Modi',
 57: 'have',
 58: 'banks',
 59: 'income',
 60: 'our',
 61: 'architecture',
 62: 'general',
 63: 'slip',
 64: 'ups',
 65: 'bank',
 66: 'registered',
 67: 'numbers',
 68: 'well',
 

Load word2vec model

In [15]:
if not w2vec:
    print("Using fine tuned . .")
    if ner:
        print("Loading "+policy+"-word2vec-ner")
        unbiased_in_domain_model = KeyedVectors.load('/Users/navreetkaur/MTP/finetune-word2vec/w2v-models/'+policy+'-word2vec-ner')
    elif blackout:
        print("Loading "+policy+"-word2vec-blackout")
        unbiased_in_domain_model = KeyedVectors.load('/Users/navreetkaur/MTP/finetune-word2vec/w2v-models/'+policy+'-word2vec-blackout')
    else:
        print("Loading "+policy+"-word2vec")
        unbiased_in_domain_model = KeyedVectors.load('Users/navreetkaur/MTP/finetune-word2vec/w2v-models/'+policy+'-word2vec')
    
word2vec_model = gensim.models.KeyedVectors.load_word2vec_format('/Users/navreetkaur/MTP/finetune-word2vec/w2v-models/GoogleNews-vectors-negative300.bin', binary=True)

Using fine tuned . .
Loading economic-word2vec-ner


Set value of the first(embedding) layer of the model

In [16]:
embed = np.zeros((len(idx2word), word2vec_model.vector_size))
for idx, word in idx2word.items():
    if not w2vec:
        try: 
            embed[idx] = unbiased_in_domain_model[word]
        except KeyError:
            try:
                embed[idx] = word2vec_model[word]
            except:
                embed[idx] = np.random.normal(size=(word2vec_model.vector_size, ))
    else:
        try:
            embed[idx] = word2vec_model[word]
        except:
            embed[idx] = np.random.normal(size=(word2vec_model.vector_size, ))
embed = torch.from_numpy(embed)
embed.shape, len(word2idx)

(torch.Size([75, 300]), 75)

In [17]:
df_train.groupby('target').size() # size of dataset and classes

target
0    2
1    2
dtype: int64

In [18]:
[pro,anti] = pickle.load(open(os.path.join('./trees',file_trees),'rb'))
[pro_test,anti_test] = pickle.load(open(os.path.join('./trees',file_trees+"_test"),'rb'))

In [19]:
CUDA=False
def Var(v):
    if CUDA: return Variable(v.cuda())
    else: return Variable(v)
trees = []
raw_words = []
vocab = []

In [25]:
for pro_tree in pro:
    pro_tree.root.set_label('pro')
for anti_tree in anti:
    anti_tree.root.set_label('anti')
for pro_tree in pro_test:
    pro_tree.root.set_label('pro')
for anti_tree in anti_test:
    anti_tree.root.set_label('anti')

## utils

In [26]:
from mytree import *
from treeUtil import *

def combine(neutral,non_neutral):
    trees = []
    trees.extend(neutral)
    trees.extend(non_neutral)
    random.shuffle(trees)
    return trees

val = {'pro':0,'anti':1,'default':-1}
# pro:1, anti:0, neutral:2 (three label)
val_all = {'pro':1,'anti':0,'default':-1,'neutral':2}
# neutral:0, non-neutral:1 (two-label)
val_neutral = {'neutral':0, 'non_neutral':1, 'default':-1}


def convert(T):
    label = val[T.label] if (hasattr(T,'label')) else None
#     print(label)
    newTree = convert_primary_new(T,label)
    annotate_all(newTree)

    return newTree

def convert_neutral(T):
    label = val_neutral[T.label] if (hasattr(T,'label')) else None
#     print(label)
    newTree = convert_primary_new(T,label)
    annotate_all(newTree)

    return newTree

def convert_all(T):
    label = val_all[T.label] if (hasattr(T,'label')) else None
#     print(label)
    newTree = convert_primary_new(T,label)
    annotate_all(newTree)

    return newTree


def convert_primary(T):
    if (hasattr(T,'label')):
        print(T.label) 
    label = val[T.label] if (hasattr(T,'label')) else None
    # label = val[T.label] if (hasattr(T,'label') ) else None # changed for ignoring neutral

    if isinstance(T,leafObj):
        newTree = Node(label,T.word,T.pos)
        newTree.isLeaf = True
        return newTree
    else:
        newTree = Node(label)
    
    leftChild = convert_primary(T.c1)
    rightChild = convert_primary(T.c2)
    leftChild.parent = newTree
    rightChild.parent = newTree

    newTree.left = leftChild
    newTree.right = rightChild

    return newTree

def convert_primary_new(T,label):
    # from IPython import embed; embed()
    if T is None:
        return None
    # label = val[T.label] if (hasattr(T,'label') ) else None # changed for ignoring neutral
    T.set_label(label)
    # if (T.isLeaf) : print (T.word)

    T.left = convert_primary_new(T.left,label)
    T.right = convert_primary_new(T.right,label)

    return T

def convertNLTK_tree_primary(tree):
    if tree.height()==2:
        newTree = Node('default',tree[0],None)
        newTree.isLeaf = True
        return newTree
    newTree = Node('default')
    leftChild = convertNLTK_tree_primary(tree[0])
    rightChild = convertNLTK_tree_primary(tree[1])
    
    leftChild.parent = newTree
    rightChild.parent = newTree

    newTree.left = leftChild
    newTree.right = rightChild

    return newTree

def convertNLTK_tree(tree):
    return Tree(convertNLTK_tree_primary(tree))

def annotate_all(T):
    if T == None: return
    if T.label != None : 
        T.annotated = True
    else:
        T.annotated = False
        T.set_label(T.parent.label)
    annotate_all(T.left)
    annotate_all(T.right)

def buildBalTree(sent):
    words = sent.split(' ')

    nodes = words

    while len(nodes)>1:
        temp = []
        for i in range(0,len(nodes),2):
            lChild = Node(None,nodes[i],None) if isinstance(nodes[i],str) else nodes[i]
            if i+1<len(nodes):
                rChild = Node(None,nodes[i+1],None) if isinstance(nodes[i+1],str) else nodes[i+1]
            else:
                rChild = None
            if isinstance(nodes[i],str):
                lChild.isLeaf = True
                if rChild is not None:
                    rChild.isLeaf = True
            newNode = Node(None)
            lChild.parent = newNode
            newNode.left = lChild
            newNode.right = rChild
            if rChild is not None:
                rChild.parent = newNode
            temp.append(newNode)
        nodes=temp
    return Tree(nodes[0])

def readFile2Trees(filename):
    trees = []
    with open(filename,'r') as file:
        for line in file:
            if line=='\n':
                continue
            else:
                [labelname,sent] = line.split(': ',1)
                tree = buildBalTree(sent)
                tree.root.set_label(val[labelname])
                if val[labelname]!=2:
                    trees.append(tree)
    return trees

In [27]:
trees = combine(pro,anti)
data = []
for i in range(len(trees)):
    data.append(Tree(convert(trees[i].root)))

In [28]:
trees_test = combine(pro_test,anti_test)
data_test = []
for i in range(len(trees_test)):
    data_test.append(Tree(convert(trees_test[i].root)))

#### Training

In [30]:
if CUDA: model = RecursiveNN(embed,word2idx).cuda()
else: model = RecursiveNN(embed,word2idx)
if economic:
    max_epochs = 100
else:
    max_epochs = 50
widgets = [progressbar.Percentage(), ' ', progressbar.Bar(), ' ', progressbar.ETA()]
l2_reg = {  'embedding.weight' : 1e-6,'W.weight' : 1e-4,'W.bias' : 1e-4,'projection.weight' : 1e-3,'projection.bias' : 1e-3}
random.shuffle(data)
trn, dev = data, data_test
len(trn), len(dev)

(4, 1)

In [31]:
logging.info('Start logging')
optimizer = torch.optim.SGD(model.parameters(), lr=0.01, momentum=0.9, dampening=0.0)
# bestAll=bestRoot=0.0
global count
count=0
BATCH_SIZE = 128
# optimizer = torch.optim.LBFGS(model.parameters(), lr=0.5, max_iter=10, history_size = 10)
bestAll=bestRoot=0.0
bestF1_0=bestF1_1=best_trn_F1_0=best_trn_F1_1=0.0
best_trn_All = best_trn_Root = 0.0 
for epoch in range(max_epochs):
#     trn = get_trn()
    print("\n\nEpoch %d" % epoch)
    logging.info('Epoch: '+str(epoch))
#     pbar = progressbar.ProgressBar(widgets=widgets, maxval=len(trn)/BATCH_SIZE).start()
    params = []
    for i in range(0,len(trn),BATCH_SIZE):
        count+=1
        batch = trn[i:min(i+BATCH_SIZE,len(trn))]
        def closure():
            global count
            optimizer.zero_grad()
            _,total_loss = model.getLoss(trn[0].root)
            for tree in batch:
                _, loss = model.getLoss(tree.root)
                total_loss += loss

            total_loss = total_loss/len(batch)
            #L2 reg
            param_dict = dict()
            for name, param in model.named_parameters():
                param_dict[name] = param.data.clone()
                if param.requires_grad:
                        total_loss += 0.5*l2_reg[name]*(torch.norm(param)**2)
            params.append(param_dict)
            print('Loss = ',total_loss.data)
            logging.info('Loss = '+str(total_loss.data))
            logger.scalar_summary('loss', total_loss.data, count)
            total_loss.backward()
            clip_grad_norm_(model.parameters(),5,2)
            return total_loss
#         pbar.update(i/BATCH_SIZE)
        optimizer.step(closure)

#     pbar.finish()

    avg_param = dict()
    for name, param1 in model.named_parameters():
            avg_param[name] = param1.data.clone()

    for i in range(1,len(params)):
        for name, param in params[i].items():
            avg_param[name] += param.clone()
    for name, param in model.named_parameters():
        if name == 'embedding.weight':
            continue
        param.data = avg_param[name]/len(params)

    correctRoot, correctAll, f1 = model.evaluate(dev)
    # correctRoot = model.eval_sent_lvl(dev,LR_clf)
    if bestAll<correctAll: bestAll=correctAll
    if bestRoot<correctRoot: bestRoot=correctRoot
    if bestF1_0<f1[0]: bestF1_0=f1[0]
    if bestF1_1<f1[1]: bestF1_1=f1[1]
    print("\nValidation All-nodes accuracy:"+str(round(correctAll,2))+"(best:"+str(round(bestAll,2))+")")
    print("Validation Root accuracy:" + str(round(correctRoot,2))+"(best:"+str(round(bestRoot,2))+")")
    print("F1:"+str([round(x,2) for x in f1])+"(best:"+str(round(bestF1_0,2))+" , "+str(round(bestF1_1,2))+")")
    logging.info("Validation All-nodes accuracy:"+str(round(correctAll,2))+"(best:"+str(round(bestAll,2))+")")
    logging.info("Validation Root accuracy:" + str(round(correctRoot,2))+"(best:"+str(round(bestRoot,2))+")")
    logging.info("F1:"+str([round(x,2) for x in f1])+"(best:"+str(round(bestF1_0,2))+" , "+str(round(bestF1_1,2))+")")
    correct_trn_Root, correct_trn_All, f1_trn = model.evaluate(trn)
    # correctRoot = model.eval_sent_lvl(dev,LR_clf)
    if best_trn_All<correct_trn_All: best_trn_All=correct_trn_All
    if best_trn_Root<correct_trn_Root: best_trn_Root=correct_trn_Root
    if best_trn_F1_0<f1_trn[0]: best_trn_F1_0=f1_trn[0]
    if best_trn_F1_1<f1_trn[1]: best_trn_F1_1=f1_trn[1]
    print("\nTraining All-nodes accuracy:"+str(round(correct_trn_All,2))+"(best:"+str(round(best_trn_All,2))+")")
    print("Training Root// accuracy:" + str(round(correct_trn_Root,2))+"(best:"+str(round(best_trn_Root,2))+")")
    print("Training F1:"+str([round(x,2) for x in f1_trn])+"(best:"+str(round(best_trn_F1_0,2))+" , "+str(round(best_trn_F1_1,2))+")")
    logging.info("Training All-nodes accuracy:"+str(round(correct_trn_All,2))+"(best:"+str(round(best_trn_All,2))+")")
    logging.info("Training Root accuracy:" + str(round(correct_trn_Root,2))+"(best:"+str(round(best_trn_Root,2))+")")
    logging.info("Training F1:"+str([round(x,2) for x in f1_trn])+"(best:"+str(round(best_trn_F1_0,2))+" , "+str(round(best_trn_F1_1,2))+")")
    info = {'valid_root_acc':correctRoot, 'valid_tree_acc':correctAll, 'train_root_acc':correct_trn_Root, 'train_tree_acc':correct_trn_All}
    for tag, value in info.items():
        logger.scalar_summary(tag,value,epoch)
    random.shuffle(trn)



Epoch 0
Loss =  tensor([30.9919])


100% |##########################################################| Time: 0:00:00
100% |##########################################################| Time: 0:00:00



Validation All-nodes accuracy:0.01(best:0.01)
Validation Root accuracy:0.0(best:0.0)
F1:[0.5, 0.02](best:0.5 , 0.02)

Training All-nodes accuracy:0.53(best:0.53)
Training Root// accuracy:0.5(best:0.5)
Training F1:[0.5, 0.67](best:0.5 , 0.67)


Epoch 1
Loss =  tensor([27.8373])


100% |##########################################################| Time: 0:00:00
100% |##########################################################| Time: 0:00:00



Validation All-nodes accuracy:0.0(best:0.01)
Validation Root accuracy:0.0(best:0.0)
F1:[0.5, 0.02](best:0.5 , 0.02)

Training All-nodes accuracy:0.55(best:0.55)
Training Root// accuracy:0.5(best:0.5)
Training F1:[0.5, 0.67](best:0.5 , 0.67)


Epoch 2
Loss =  tensor([27.7548])


...

100% |##########################################################| Time: 0:00:00
100% |##########################################################| Time: 0:00:00



Training All-nodes accuracy:0.94(best:0.94)
Training Root// accuracy:1.0(best:1.0)
Training F1:[1.0, 1.0](best:1.0 , 1.0)


Epoch 98
Loss =  tensor([7.2494])

Validation All-nodes accuracy:0.36(best:0.79)
Validation Root accuracy:0.0(best:1.0)
F1:[0.5, 0.02](best:0.99 , 0.5)


100% |##########################################################| Time: 0:00:00
100% |##########################################################| Time: 0:00:00



Training All-nodes accuracy:0.94(best:0.94)
Training Root// accuracy:1.0(best:1.0)
Training F1:[1.0, 1.0](best:1.0 , 1.0)


Epoch 99
Loss =  tensor([7.3365])

Validation All-nodes accuracy:0.36(best:0.79)
Validation Root accuracy:0.0(best:1.0)
F1:[0.5, 0.02](best:0.99 , 0.5)



Training All-nodes accuracy:0.94(best:0.94)
Training Root// accuracy:1.0(best:1.0)
Training F1:[1.0, 1.0](best:1.0 , 1.0)


In [32]:
namecode

'economic_w2v_1ProAnti_0General_1NER_0Blackout_1Balance_1Undersample_1Fixed'

In [33]:
import pickle
pickle.dump(model,open("./models/"+namecode+'.pkl','wb'))

#### Evaluation

In [35]:
model = pickle.load(open("/Users/navreetkaur/MTP/tech_classifier/models/"+namecode+'.pkl','rb'))
correctRoot, correctAll, f1 = model.evaluate(data_test)
print("Validation all nodes accuracy: ", correctAll)
print("Validation root accuracy: ", correctRoot)
print("Validation root accuracy: ", f1)

Validation all nodes accuracy:  0.36
Validation root accuracy:  0.0
Validation root accuracy:  [0.5, 0.019230769230769232]
